<a href="https://colab.research.google.com/github/anandt555/BigData-Practice/blob/main/squared_error_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### EDL with mnist with Custom loss(squared error + KL Divergence Term)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Download MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=1000, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

# Define custom loss function and KL divergence
import torch.nn.functional as F
import numpy as np

def KL(alpha, num_classes=10):
    one = torch.ones((1, num_classes), dtype=torch.float32)
    S = torch.sum(alpha, dim=1, keepdim=True)

    kl = torch.lgamma(S) - torch.sum(torch.lgamma(alpha), dim=1, keepdim=True) + \
         torch.sum(torch.lgamma(one), dim=1, keepdim=True) - torch.lgamma(torch.sum(one, dim=1, keepdim=True)) + \
         torch.sum((alpha - one) * (torch.digamma(alpha) - torch.digamma(S)), dim=1, keepdim=True)

    return kl

def custom_loss(y_true, output):
    epochs = [1]

    y_evidence = F.relu(output)
    alpha = y_evidence + 1
    S = torch.sum(alpha, dim=1, keepdim=True)
    p = alpha / S

    err = torch.sum(torch.pow((y_true - p), 2), dim=1, keepdim=True)
    var = torch.sum(alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)

    l = torch.sum(err + var, dim=1, keepdim=True)

    kl = torch.min(torch.tensor(1.0), torch.tensor(epochs[0] / 50)) * torch.sum(KL((1 - y_true) * (alpha) + y_true))
    return torch.sum(l + kl)

# LeNet model with Dirichlet distribution for Evidential Deep Learning
class LeNetDirichlet(nn.Module):
    def __init__(self):
        super(LeNetDirichlet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5, stride=1, padding=0)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        alpha = torch.abs(x) + 1  # Use absolute values for simplicity, adjust as needed

        u = 10 / torch.sum(alpha, dim=1, keepdim=True)

        prob = alpha / torch.sum(alpha, 1, keepdim=True)

        return prob, u, alpha

# Train LeNet model with Dirichlet distribution for Evidential Deep Learning
lenet_dirichlet = LeNetDirichlet()
optimizer_dirichlet = optim.Adam(lenet_dirichlet.parameters(), lr=0.001)

for epoch in range(10):
    lenet_dirichlet.train()
    for inputs, labels in train_loader:
        optimizer_dirichlet.zero_grad()
        outputs, _, _ = lenet_dirichlet(inputs)
        loss = custom_loss(F.one_hot(labels, num_classes=10).float(), outputs)  # Using custom loss
        loss.backward()
        optimizer_dirichlet.step()

    lenet_dirichlet.eval()
    correct_train = 0
    total_train = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs, uncertainty_train, _ = lenet_dirichlet(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs, uncertainty_test, _ = lenet_dirichlet(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    print('epoch %d - training accuracy: %2.4f \t training uncertainty: %2.4f \t testing accuracy: %2.4f \t testing uncertainty: %2.4f' %
          (epoch+1, correct_train / total_train, uncertainty_train.mean(), correct_test / total_test, uncertainty_test.mean()))


epoch 1 - training accuracy: 0.9348 	 training uncertainty: 0.4443 	 testing accuracy: 0.9426 	 testing uncertainty: 0.4405
epoch 2 - training accuracy: 0.9641 	 training uncertainty: 0.2648 	 testing accuracy: 0.9678 	 testing uncertainty: 0.2562
epoch 3 - training accuracy: 0.9740 	 training uncertainty: 0.2004 	 testing accuracy: 0.9765 	 testing uncertainty: 0.1908
epoch 4 - training accuracy: 0.9799 	 training uncertainty: 0.1740 	 testing accuracy: 0.9809 	 testing uncertainty: 0.1600
epoch 5 - training accuracy: 0.9833 	 training uncertainty: 0.1423 	 testing accuracy: 0.9828 	 testing uncertainty: 0.1336
epoch 6 - training accuracy: 0.9858 	 training uncertainty: 0.1347 	 testing accuracy: 0.9853 	 testing uncertainty: 0.1195
epoch 7 - training accuracy: 0.9884 	 training uncertainty: 0.1159 	 testing accuracy: 0.9873 	 testing uncertainty: 0.1086
epoch 8 - training accuracy: 0.9895 	 training uncertainty: 0.1127 	 testing accuracy: 0.9882 	 testing uncertainty: 0.0980
epoch 9 

### EDL + mixup with mnist with Custom loss(squared error + KL Divergence Term)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np

# Download MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=1000, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

# Define custom loss function and KL divergence
import torch.nn.functional as F

def KL(alpha, num_classes=10):
    one = torch.ones((1, num_classes), dtype=torch.float32)
    S = torch.sum(alpha, dim=1, keepdim=True)

    kl = torch.lgamma(S) - torch.sum(torch.lgamma(alpha), dim=1, keepdim=True) + \
         torch.sum(torch.lgamma(one), dim=1, keepdim=True) - torch.lgamma(torch.sum(one, dim=1, keepdim=True)) + \
         torch.sum((alpha - one) * (torch.digamma(alpha) - torch.digamma(S)), dim=1, keepdim=True)

    return kl

def custom_loss(y_true, output):
    epochs = [1]

    y_evidence = F.relu(output)
    alpha = y_evidence + 1
    S = torch.sum(alpha, dim=1, keepdim=True)
    p = alpha / S

    err = torch.sum(torch.pow((y_true - p), 2), dim=1, keepdim=True)
    var = torch.sum(alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)

    l = torch.sum(err + var, dim=1, keepdim=True)

    kl = torch.min(torch.tensor(1.0), torch.tensor(epochs[0] / 50)) * torch.sum(KL((1 - y_true) * (alpha) + y_true))
    return torch.sum(l + kl)

# Mixup function for Evidential Deep Learning
def mixup_data(x, y, alpha=0.1):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    # Convert labels to one-hot encoding
    y_onehot = torch.zeros(y.size(0), 10).to(x.device)
    y_onehot.scatter_(1, y.view(-1, 1).long(), 1)
    mixed_y = lam * y_onehot + (1 - lam) * y_onehot[index, :]
    mixed_y = mixed_y.argmax(dim=1)
    return mixed_x, mixed_y

# LeNet model with Dirichlet distribution for Evidential Deep Learning
class LeNetDirichlet(nn.Module):
    def __init__(self):
        super(LeNetDirichlet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5, stride=1, padding=0)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        alpha = torch.abs(x) + 1  # Use absolute values for simplicity, adjust as needed

        u = 10 / torch.sum(alpha, dim=1, keepdim=True)

        prob = alpha / torch.sum(alpha, 1, keepdim=True)

        return prob, u, alpha

# Train LeNet model with Dirichlet distribution for Evidential Deep Learning
lenet_dirichlet = LeNetDirichlet()
optimizer_dirichlet = optim.Adam(lenet_dirichlet.parameters(), lr=0.001)

for epoch in range(10):
    lenet_dirichlet.train()
    for inputs, labels in train_loader:
        inputs, labels = mixup_data(inputs, labels.unsqueeze(1).float())  # Applying mixup

        optimizer_dirichlet.zero_grad()
        outputs, _, _ = lenet_dirichlet(inputs)
        loss = custom_loss(F.one_hot(labels, num_classes=10).float(), outputs)  # Using custom loss
        loss.backward()
        optimizer_dirichlet.step()

    lenet_dirichlet.eval()
    correct_train = 0
    total_train = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs, uncertainty_train, _ = lenet_dirichlet(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs, uncertainty_test, _ = lenet_dirichlet(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    print('epoch %d - training accuracy: %2.4f \t training uncertainty: %2.4f \t testing accuracy: %2.4f \t testing uncertainty: %2.4f' %
          (epoch+1, correct_train / total_train, uncertainty_train.mean(), correct_test / total_test, uncertainty_test.mean()))

epoch 1 - training accuracy: 0.9460 	 training uncertainty: 0.4117 	 testing accuracy: 0.9525 	 testing uncertainty: 0.4051
epoch 2 - training accuracy: 0.9666 	 training uncertainty: 0.2740 	 testing accuracy: 0.9708 	 testing uncertainty: 0.2641
epoch 3 - training accuracy: 0.9747 	 training uncertainty: 0.2210 	 testing accuracy: 0.9781 	 testing uncertainty: 0.2047
epoch 4 - training accuracy: 0.9795 	 training uncertainty: 0.1997 	 testing accuracy: 0.9813 	 testing uncertainty: 0.1913
epoch 5 - training accuracy: 0.9824 	 training uncertainty: 0.1661 	 testing accuracy: 0.9843 	 testing uncertainty: 0.1552
epoch 6 - training accuracy: 0.9838 	 training uncertainty: 0.1484 	 testing accuracy: 0.9857 	 testing uncertainty: 0.1378
epoch 7 - training accuracy: 0.9861 	 training uncertainty: 0.1580 	 testing accuracy: 0.9875 	 testing uncertainty: 0.1494
epoch 8 - training accuracy: 0.9871 	 training uncertainty: 0.1320 	 testing accuracy: 0.9876 	 testing uncertainty: 0.1237
epoch 9 

### EDL + regmixup with mnist with Custom loss(squared error + KL Divergence Term)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np

# Download MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=1000, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

# Define custom loss function and KL divergence
import torch.nn.functional as F

def KL(alpha, num_classes=10):
    one = torch.ones((1, num_classes), dtype=torch.float32)
    S = torch.sum(alpha, dim=1, keepdim=True)

    kl = torch.lgamma(S) - torch.sum(torch.lgamma(alpha), dim=1, keepdim=True) + \
         torch.sum(torch.lgamma(one), dim=1, keepdim=True) - torch.lgamma(torch.sum(one, dim=1, keepdim=True)) + \
         torch.sum((alpha - one) * (torch.digamma(alpha) - torch.digamma(S)), dim=1, keepdim=True)

    return kl

def custom_loss(y_true, output):
    epochs = [1]

    y_evidence = F.relu(output)
    alpha = y_evidence + 1
    S = torch.sum(alpha, dim=1, keepdim=True)
    p = alpha / S

    err = torch.sum(torch.pow((y_true - p), 2), dim=1, keepdim=True)
    var = torch.sum(alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)

    l = torch.sum(err + var, dim=1, keepdim=True)

    kl = torch.min(torch.tensor(1.0), torch.tensor(epochs[0] / 50)) * torch.sum(KL((1 - y_true) * (alpha) + y_true))
    return torch.sum(l + kl)


def regmixup(x, y, beta=1.0, cutmix_min=0.0, cutmix_max=0.8):
    lambda_ = np.random.beta(beta, beta)
    r = np.random.rand(1)
    if r < cutmix_min:
        return x, y
    elif r > cutmix_max:
        return x.clone(), y.clone()

    # CutMix regularization (optional)
    indices = torch.randperm(x.shape[0])  # Shuffle for random mask generation
    mask = torch.ones(x.size()).float()
    bbx_y = np.random.randint(0, x.size(-2) - int(cutmix_max * x.size(-2)))
    bbx_x = np.random.randint(0, x.size(-1) - int(cutmix_max * x.size(-1)))
    bbx_y2 = bbx_y + int(cutmix_max * x.size(-2))
    bbx_x2 = bbx_x + int(cutmix_max * x.size(-1))
    mask[indices, :, bbx_y:bbx_y2, bbx_x:bbx_x2] = 0

    mixed_x = lambda_ * x * mask + (1 - lambda_) * x[indices] * (1 - mask)

    # Mixed label using weighted average
    mixed_y = lambda_ * y + (1 - lambda_) * y[indices]

    return mixed_x, mixed_y


# LeNet model with Dirichlet distribution for Evidential Deep Learning
class LeNetDirichlet(nn.Module):
    def __init__(self):
        super(LeNetDirichlet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5, stride=1, padding=0)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        alpha = torch.abs(x) + 1  # Use absolute values for simplicity, adjust as needed

        u = 10 / torch.sum(alpha, dim=1, keepdim=True)

        prob = alpha / torch.sum(alpha, 1, keepdim=True)

        return prob, u, alpha

# Train LeNet model with Dirichlet distribution for Evidential Deep Learning
lenet_dirichlet = LeNetDirichlet()
optimizer_dirichlet = optim.Adam(lenet_dirichlet.parameters(), lr=0.001)

for epoch in range(10):
    lenet_dirichlet.train()
    for inputs, labels in train_loader:
        labels_onehot = F.one_hot(labels, num_classes=10).float()
        inputs, labels_onehot = regmixup(inputs, labels_onehot)  # Applying mixup

        optimizer_dirichlet.zero_grad()
        outputs, _, _ = lenet_dirichlet(inputs)
        loss = custom_loss(labels_onehot, outputs)  # Using custom loss
        loss.backward()
        optimizer_dirichlet.step()

    lenet_dirichlet.eval()
    correct_train = 0
    total_train = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs, uncertainty_train, _ = lenet_dirichlet(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()  # Fix dimension here

    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs, uncertainty_test, _ = lenet_dirichlet(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()  # Fix dimension here

    print('epoch %d - training accuracy: %2.4f \t training uncertainty: %2.4f \t testing accuracy: %2.4f \t testing uncertainty: %2.4f' %
          (epoch+1, correct_train / total_train, uncertainty_train.mean(), correct_test / total_test, uncertainty_test.mean()))

epoch 1 - training accuracy: 0.9070 	 training uncertainty: 0.5979 	 testing accuracy: 0.9108 	 testing uncertainty: 0.5910
epoch 2 - training accuracy: 0.9539 	 training uncertainty: 0.5339 	 testing accuracy: 0.9579 	 testing uncertainty: 0.5261
epoch 3 - training accuracy: 0.9650 	 training uncertainty: 0.4187 	 testing accuracy: 0.9685 	 testing uncertainty: 0.4055
epoch 4 - training accuracy: 0.9710 	 training uncertainty: 0.3606 	 testing accuracy: 0.9733 	 testing uncertainty: 0.3449
epoch 5 - training accuracy: 0.9758 	 training uncertainty: 0.3616 	 testing accuracy: 0.9780 	 testing uncertainty: 0.3441
epoch 6 - training accuracy: 0.9786 	 training uncertainty: 0.3318 	 testing accuracy: 0.9810 	 testing uncertainty: 0.3166
epoch 7 - training accuracy: 0.9815 	 training uncertainty: 0.3298 	 testing accuracy: 0.9841 	 testing uncertainty: 0.3139
epoch 8 - training accuracy: 0.9831 	 training uncertainty: 0.3069 	 testing accuracy: 0.9851 	 testing uncertainty: 0.2851
epoch 9 